In [7]:
import pandas as pd
import numpy as np
from datetime import timedelta

# Read your cleaned CRRL data
df = pd.read_csv('../CRRL_subset.csv')  # Replace with your actual file name
print(df.head())

# Remove rows where the timestamp column contains "Timestamp"
df = df[df['UTCTimestampCollected'] != 'Timestamp'].reset_index(drop=True)

# Then convert to datetime
df['UTCTimestampCollected'] = pd.to_datetime(df['UTCTimestampCollected'])
df = df.sort_values('UTCTimestampCollected').reset_index(drop=True)

print(f"Analyzing continuity of data with {len(df)} records")
print(f"Expected interval: 30 minutes")
print("-" * 60)

# Calculate time differences between consecutive rows
df['time_diff'] = df['UTCTimestampCollected'].diff()

# Define what constitutes a gap (more than 30 minutes)
expected_interval = timedelta(minutes=30)
tolerance = timedelta(minutes=5)  # Allow 5-minute tolerance
gap_threshold = expected_interval + tolerance

# Find gaps (where time difference > 35 minutes)
gaps = df[df['time_diff'] > gap_threshold].copy()

# Create a list to store continuous periods
continuous_periods = []
gap_records = []

if len(gaps) == 0:
    # No gaps found - entire dataset is continuous
    continuous_periods.append({
        'period': 1,
        'start_date': df['UTCTimestampCollected'].iloc[0],
        'end_date': df['UTCTimestampCollected'].iloc[-1],
        'duration_hours': (df['UTCTimestampCollected'].iloc[-1] - df['UTCTimestampCollected'].iloc[0]).total_seconds() / 3600,
        'record_count': len(df)
    })
else:
    # Process continuous periods between gaps
    period_start_idx = 0

    for i, (gap_idx, gap_row) in enumerate(gaps.iterrows()):
        # End of current continuous period (row before the gap)
        period_end_idx = gap_idx - 1

        if period_end_idx >= period_start_idx:
            period_start = df.iloc[period_start_idx]['UTCTimestampCollected']
            period_end = df.iloc[period_end_idx]['UTCTimestampCollected']
            duration_hours = (period_end - period_start).total_seconds() / 3600
            record_count = period_end_idx - period_start_idx + 1

            continuous_periods.append({
                'period': len(continuous_periods) + 1,
                'start_date': period_start,
                'end_date': period_end,
                'duration_hours': duration_hours,
                'record_count': record_count
            })

        # Record the gap
        gap_start = df.iloc[gap_idx - 1]['UTCTimestampCollected'] if gap_idx > 0 else None
        gap_end = gap_row['UTCTimestampCollected']
        gap_duration = gap_row['time_diff']

        gap_records.append({
            'gap': len(gap_records) + 1,
            'gap_start': gap_start,
            'gap_end': gap_end,
            'gap_duration': gap_duration,
            'missing_records_est': int(gap_duration.total_seconds() / (30 * 60)) - 1
        })

        # Start of next continuous period
        period_start_idx = gap_idx

    # Handle the last continuous period after the final gap
    if period_start_idx < len(df):
        period_start = df.iloc[period_start_idx]['UTCTimestampCollected']
        period_end = df.iloc[-1]['UTCTimestampCollected']
        duration_hours = (period_end - period_start).total_seconds() / 3600
        record_count = len(df) - period_start_idx

        continuous_periods.append({
            'period': len(continuous_periods) + 1,
            'start_date': period_start,
            'end_date': period_end,
            'duration_hours': duration_hours,
            'record_count': record_count
        })

# Create DataFrames for analysis
continuous_df = pd.DataFrame(continuous_periods)
gaps_df = pd.DataFrame(gap_records)

# Print results
print(f"CONTINUOUS DATA PERIODS ({len(continuous_periods)} found):")
print("=" * 60)

for _, period in continuous_df.iterrows():
    print(f"Period {period['period']}:")
    print(f"  Start: {period['start_date']}")
    print(f"  End: {period['end_date']}")
    print(f"  Duration: {period['duration_hours']:.1f} hours ({period['duration_hours']/24:.1f} days)")
    print(f"  Records: {period['record_count']}")
    print()

if len(gaps_df) > 0:
    print(f"DATA GAPS ({len(gaps_df)} found):")
    print("=" * 60)

    for _, gap in gaps_df.iterrows():
        print(f"Gap {gap['gap']}:")
        print(f"  Before: {gap['gap_start']}")
        print(f"  After: {gap['gap_end']}")
        print(f"  Duration: {gap['gap_duration']}")
        print(f"  Estimated missing records: {gap['missing_records_est']}")
        print()

# Save results to files
continuous_df.to_csv('continuous_periods.csv', index=False)
gaps_df.to_csv('data_gaps.csv', index=False)

print("Results saved to:")
print("  - continuous_periods.csv")
print("  - data_gaps.csv")

# Summary statistics
total_data_hours = continuous_df['duration_hours'].sum()
total_gap_hours = gaps_df['gap_duration'].dt.total_seconds().sum() / 3600 if len(gaps_df) > 0 else 0
total_time_span = (df['UTCTimestampCollected'].max() - df['UTCTimestampCollected'].min()).total_seconds() / 3600

print(f"\nSUMMARY:")
print(f"  Total time span: {total_time_span:.1f} hours ({total_time_span/24:.1f} days)")
print(f"  Continuous data: {total_data_hours:.1f} hours ({total_data_hours/24:.1f} days)")
print(f"  Gap time: {total_gap_hours:.1f} hours ({total_gap_hours/24:.1f} days)")
print(f"  Data completeness: {(total_data_hours/total_time_span)*100:.1f}%")

# Find longest continuous period
if len(continuous_df) > 0:
    longest_period = continuous_df.loc[continuous_df['duration_hours'].idxmax()]
    print(f"  Longest continuous period: {longest_period['duration_hours']:.1f} hours ({longest_period['duration_hours']/24:.1f} days)")
    print(f"    From: {longest_period['start_date']}")
    print(f"    To: {longest_period['end_date']}")

   Unnamed: 0 NetSiteAbbrev          County UTCTimestampCollected  \
0           0    Station ID  Station County             Timestamp   
1      105374          CRRL         Carroll   2019-05-01 22:00:00   
2      105380          CRRL         Carroll   2019-05-01 22:30:00   
3      105386          CRRL         Carroll   2019-05-01 23:00:00   
4      105392          CRRL         Carroll   2019-05-01 23:30:00   

                   TAIR           DWPT                PRCP           PRES  \
0  Air Temperature (°C)  Dewpoint (°C)  Precipitation (mm)  Pressure (mb)   
1                22.218        19.1653                0.71         999.75   
2               24.2952        19.5039                 0.0        999.601   
3               21.4553        19.4506                 0.0        999.672   
4               20.5968        17.6376                 0.0        1000.14   

                    RELH                    SRAD  ...  \
0  Relative Humidity (%)  Solar Radiation (W/m²)  ...   
1       

/var/folders/x3/czpqf5ks36d7yf5gcqbkp2140000gn/T/ipykernel_9702/733955020.py:6: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../CRRL_subset.csv')  # Replace with your actual file name
